In [1]:
from mne.time_frequency import psd_array_multitaper, psd_array_welch
from scipy.signal import hilbert
from scipy.signal import welch
import multiprocessing as mp
from scipy.io import loadmat
from scipy.io import savemat
from scipy import signal
from fooof import FOOOF
import edgeofpy as eop
from scipy import stats
import pandas as pd
import numpy as np
import powerlaw
import argparse
import pickle
import mne.io
import mne
import os
import glob


In [ ]:

out_dir = 'data/output/AVC/'
in_dir = 'data/input/continuous/'

FIL_FREQ = (1, 40) # bandpass frequencies
THRESH_TYPE = 'both' # Fosque22: 'both'
# GAMMA_EXPONENT_RANGE = (0, 2) #NOT USED CURRENTLY
# LATTICE_SEARCH_STEP = 0.1 #NOT USED CURRENTLY
BIN_THRESHOLD = float(2)
MAX_IEI = float(0.008)
BRANCHING_RATIO_TIME_BIN = float(0.008)
#
#  The BIN_THRESHOLD is too high (detecting very few events).
# The MAX_IEI is too strict, discarding short events.

# output
out = {'mean_iei':[],
        'tau':[],
        'tau_dist':[],
        'tau_dist_TR':[],
        'alpha':[],
        'alpha_dist':[],
        'alpha_dist_TR':[],
        'third':[],
        'dcc_cn':[],
        'avl_br':[],
        'br':[],
        'rep_dissimilarity_avg':[],
        'rep_size':[],
        'fano':[],
        'chi_test':[],
        'chi_notest':[],
        'sig_length':[],
        'len_avls':[],
        'data_mean':[],
        'data_std':[]
        }

# make output directory
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# load patient info and conditions
# info = pd.read_csv(args.part_info,sep = ',', index_col=None)
# P_IDS = info['ID']
# Cond = info['Cond']
# Drug = info['Drug']

paths = glob.glob(in_dir+'*.fif')
paths.sort()

#loop over all conditions and particiants
# for i, p_id in enumerate(P_IDS):
for path_i, path in enumerate(paths):
    sub = os.path.basename(path)[:4]
    day = os.path.basename(path)[5:9]
    condition = os.path.basename(path)[10:-8]

    print(f"Analyzing Avlanches of {sub} {day} {condition}");

    #################################
    #          LOAD  DATA          #
    #################################

    
    # data = loadmat(input_fname)
    # data = data['dataOnlyGoodDataPoints']
    # fs = 1450
    # sig_length = min(data.shape[1]/fs , 300)
    # nr_channels =  data.shape[0]

    # # cut data and only use first 5 min or less
    # cut = np.int(sig_length*fs)
    # data = data[:,:cut]

    # data_mean = np.mean(np.abs(data))
    # data_std = np.std(data)

    # data_filt = mne.filter.filter_data(data, sfreq=fs, l_freq=FIL_FREQ[0], h_freq=FIL_FREQ[1],verbose=False)

    # events_by_chan = eop.binarized_events(data_filt, threshold=BIN_THRESHOLD,
    #                             thresh_type=THRESH_TYPE, null_value=0)
    # events_one_chan = np.sum(events_by_chan, axis=0)


    raw = mne.io.read_raw_fif(path, preload=True)
    raw.interpolate_bads(reset_bads=True)
    #raw.pick_types(eeg=True, meg=False, stim=False, eog=False, ecg=False, emg=False, misc=False, exclude='bads')

    data = raw.get_data()[:32,:]
    fs= 256
    sig_length = min(data.shape[1]/fs , 300)
    nr_channels =  data.shape[0]

    # cut data and only use first 5 min or less
    cut = np.int(sig_length*fs)
    data = data[:,:cut]

    data_mean = np.mean(np.abs(data))
    data_std = np.std(data)

    data_filt = mne.filter.filter_data(data, sfreq=fs, l_freq=FIL_FREQ[0], h_freq=FIL_FREQ[1],verbose=False)

    events_by_chan = eop.binarized_events(data_filt, threshold=BIN_THRESHOLD,
                                thresh_type=THRESH_TYPE, null_value=0)
    events_one_chan = np.sum(events_by_chan, axis=0)


    #################################
    #    Avalanches                 #
    #################################

    # Detect avalanches
    #breakpoint()
    avls, _, _, mean_iei = eop.detect_avalanches(events_by_chan, fs,
                                                    max_iei=MAX_IEI,
                                                    threshold=BIN_THRESHOLD,
                                                    thresh_type=THRESH_TYPE)

    sizes = [x['size'] for x in avls]
    dur_bin = [x['dur_bin'] for x in avls]
    dur_sec = [x['dur_sec'] for x in avls]
    len_avls = len(avls)
    # save Avalanches
    avls_out = f'{out_dir}/AVC_bin_{BIN_THRESHOLD}_iei_{MAX_IEI}/'
    os.makedirs(avls_out,exist_ok = True)
    with open(f'{avls_out}Avalanches_{sub}_{day}_{condition}.p', 'wb') as f:
            pickle.dump(avls, f)

    #################################
    #    TAU                 #
    #################################
    # Estimate fit and extract exponents with min and max of data

    size_fit = eop.fit_powerlaw(sizes, xmin=1, discrete = True, xmax = None)
    tau = size_fit['power_law_exp']
    tau_dist = size_fit['best_fit']
    tau_dist_TR = size_fit['T_R_sum']


    #################################
    #    ALPHA                     #
    #################################

    #dur_bin_fit = eop.fit_powerlaw(dur_bin, discrete = True)
    #alpha_bin = dur_bin_fit['power_law_exp']

    dur_fit = eop.fit_powerlaw(dur_sec, xmin='min', xmax = None, discrete = False)
    alpha = dur_fit['power_law_exp']
    alpha_dist = dur_fit['best_fit']
    alpha_dist_TR = dur_fit['T_R_sum']


    #################################
    #    Third   and DCC            #
    #################################

    #third_bin = eop.fit_third_exponent(sizes, dur_bin, discrete= True)
    third = eop.fit_third_exponent(sizes, dur_sec, discrete= False, method = 'pl')

    #dcc_cn_bin = eop.dcc(tau, alpha_bin, third_bin)
    dcc_cn = eop.dcc(tau, alpha, third)


    #################################
    #    REPERTPOIRE               #
    #################################

    # Estimate avalanche functional repertoire
    repertoire = eop.avl_repertoire(avls)
    # normalize the repertoire by signal length
    rep_size = repertoire.shape[0]/sig_length

    #rep_similarity_mat = eop.avl_pattern_similarity(repertoire, norm=True)
    rep_similarity_mat = eop.avl_pattern_dissimilarity(repertoire, norm=True)
    rep_dissimilarity_avg = np.mean(rep_similarity_mat)

    #################################
    #    Branching Ratio            #
    #################################
    # Calculate avalanche branching ratio
    avl_br = eop.avl_branching_ratio(avls)
    # Calculate branching ratio
    br = eop.branching_ratio(events_one_chan, BRANCHING_RATIO_TIME_BIN, fs)


    #################################
    #   Susceptibility              #
    #################################

    # Calculate Fano factor
    fano = eop.fano_factor(events_one_chan)

    # Calculate susceptibility
    chi_test, _ = eop.susceptibility(events_by_chan,test = True)
    chi_notest, _ = eop.susceptibility(events_by_chan,test = False)

    ## Save output
    for name in out.keys():
        out[name].append(locals()[name])
    output_df = pd.DataFrame(out)

    if path_i == 0:
        output_2 = pd.DataFrame(columns=['sub', 'day', 'condition'])
    output_2 = output_2.append({'sub': sub, 'day': day, 'condition': condition}, ignore_index=True)

tosave = pd.concat((output_2,output_df),axis = 1)
tosave.to_csv(f'{out_dir}/AVC_bin_{BIN_THRESHOLD}_iei_{MAX_IEI}.csv', index=False, sep=',')

Analyzing Avlanches of sub0 day2 jhana
Opening raw data file data/input/continuous/sub0-day2-jhana-raw.fif...
    Range : 512 ... 323799 =      2.000 ...  1264.840 secs
Ready.
Reading 0 ... 323287  =      0.000 ...  1262.840 secs...


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_26237/1401125018.py:104: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cut = np.int(sig_length*fs)
Assuming nested distributions


Analyzing Avlanches of sub0 day2 mindfulness
Opening raw data file data/input/continuous/sub0-day2-mindfulness-raw.fif...
    Range : 512 ... 294535 =      2.000 ...  1150.527 secs
Ready.
Reading 0 ... 294023  =      0.000 ...  1148.527 secs...


/Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/venv/lib/python3.8/site-packages/powerlaw.py:1615: RuntimeWarning: invalid value encountered in true_divide
  CDF = CDF/norm
'nan' in fit cumulative distribution values.
Likely underflow or overflow error: the optimal fit for this distribution gives values that are so extreme that we lack the numerical precision to calculate them.
Assuming nested distributions
/Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/venv/lib/python3.8/site-packages/edgeofpy/avalanche.py:702: RuntimeWarning: divide by zero encountered in true_divide
  br = np.ma.masked_invalid(events[1:] / events[:-1]).mean()
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_26237/1401125018.py:213: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_2 = output_2.append({'sub': sub, 'day': day, 'condition': condition}, ignore_index=True)
/var/folders/

## re load avalanches

In [3]:
avl_file = "data/output/AVC/AVC_bin_2.0_iei_0.008/Avalanches_sub0_day2_jhana.p"
with open(avl_file, "rb") as f:
    avalanches = pickle.load(f)

# Check the structure of the loaded data
print(f"Number of avalanches: {len(avalanches)}")
print("First avalanche example:", avalanches[0])  # Print the first avalanche event


Number of avalanches: 5488
First avalanche example: {'start_time': 0.0078125, 'end_time': 0.01953125, 'size': 18, 'dur_sec': 0.01171875, 'dur_bin': 2, 'n_chan': 18, 'profile': array([17.,  1.]), 'pattern': array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1])}
